In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset

import numpy as np

import math

import time

import dataloader
import models
import training_fun

import optuna

import joblib

import pygad

import HydroErr

In [24]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SEQ_LENGTH = 365 * 2
TARGET_SEQ_LENGTH = 365
BASE_LENGTH = SEQ_LENGTH - TARGET_SEQ_LENGTH

FORCING_DIM = 3

N_CATCHMENTS = 559

# training hyperparameters
EPOCHS = 500
TRAIN_YEAR = 19
PATIENCE = 20

use_amp = True
compile_model = False

if compile_model:
    torch.set_float32_matmul_precision("high")

memory_saving = False
if memory_saving:
    storge_device = "cpu"
    computing_device = DEVICE
    VAL_STEPS = 500
else:
    storge_device = DEVICE
    computing_device = DEVICE

In [25]:
embedding = torch.load("data/lstm_embedding.pt", map_location=torch.device('cpu')).to(computing_device)
decoder = torch.load("data/lstm_decoder.pt", map_location=torch.device('cpu')).to(computing_device)

embedding.eval()
decoder.eval()

# dimension of embedding
catchment_embeddings=[x.data for x in embedding.parameters()][0]
LATENT_dim = catchment_embeddings.shape[1]

In [26]:
dtrain_val = dataloader.Forcing_Data(
    "data/camels_train_val.csv",
    record_length=3652,
    storge_device=storge_device,
    seq_length=SEQ_LENGTH,
    target_seq_length=TARGET_SEQ_LENGTH,
    base_length=BASE_LENGTH,
)

# dtrain = dataloader.Forcing_Data(
#     "camels_train.csv",
#     record_length=2922,
#     storge_device=storge_device,
#     seq_length=SEQ_LENGTH,
#     target_seq_length=TARGET_SEQ_LENGTH,
#     base_length=BASE_LENGTH,
# )

# dval = dataloader.Forcing_Data(
#     "camels_val.csv",
#     record_length=1095,
#     storge_device=storge_device,
#     seq_length=SEQ_LENGTH,
#     target_seq_length=TARGET_SEQ_LENGTH,
#     base_length=BASE_LENGTH,
# )

dtest = dataloader.Forcing_Data(
    "data/camels_test.csv",
    record_length=4383,
    storge_device=storge_device,
    seq_length=SEQ_LENGTH,
    target_seq_length=TARGET_SEQ_LENGTH,
    base_length=BASE_LENGTH,
)

In [27]:
class Objective_builder:
    def __init__(self, x, y, eval_fun):
        self.eval_fun = eval_fun
        self.x = x.contiguous()
        self.y = y.contiguous()
    
    def eval(self, code, return_summary = True):
        
        # numpy to torch tensor
        code = torch.from_numpy(code).unsqueeze(0).to(dtype=torch.float32).to(computing_device)
        code = code.expand(self.x.shape[0], -1)
        
        # BASE_LENGTH is from global
        pred = decoder.decode(code, self.x).view(-1).detach().cpu().numpy()

        ob = self.y.view(-1).detach().cpu().numpy()
        
        if return_summary:
          gof = self.eval_fun(simulated_array=pred, observed_array=ob)
          return gof
        else:
          return pred, ob

In [28]:
num_generations = 200
num_parents_mating = 10

sol_per_pop = 100
num_genes = LATENT_dim

init_range_low = catchment_embeddings.detach().cpu().min().numpy().tolist()
init_range_high = catchment_embeddings.detach().cpu().max().numpy().tolist()

parent_selection_type = "sss"

crossover_type = "single_point"

mutation_type = "random"
mutation_probability = 0.25

x_batch_train_val, y_batch_train_val = dtrain_val.get_val_batch()
x_batch_test, y_batch_test = dtest.get_val_batch()

def evaluate_calibration(selected_catchment=0):
    
    x = x_batch_train_val[:,selected_catchment,:,:]
    y = y_batch_train_val[:,selected_catchment,:]

    x, y = x.to(computing_device), y.to(computing_device)

    fn = Objective_builder(x,y,HydroErr.kge_2009)

    def fitness_func(solution, solution_idx):
        return fn.eval(solution)

    ga_instance = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_func,
                        sol_per_pop=sol_per_pop,
                        num_genes=num_genes,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_probability = mutation_probability,
                        stop_criteria=["saturate_10"])

    ga_instance.run()

    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    
    # evaluate on test dataset
    x = x_batch_test[:,selected_catchment,:,:]
    y = y_batch_test[:,selected_catchment,:]

    x, y = x.to(computing_device), y.to(computing_device)

    fn = Objective_builder(x,y,HydroErr.kge_2009)

    return fn.eval(solution)

In [29]:
N_CATCHMENTS = 559
calibrated_KGES = np.ones(N_CATCHMENTS)

for i in range(N_CATCHMENTS):
    print(f'i={i} starts')
    calibrated_KGES[i] = evaluate_calibration(i)
    print(f'fit={calibrated_KGES[i]}')

i=0 starts
fit=0.9034413918025028
i=1 starts
fit=0.8433364951383684
i=2 starts
fit=0.8974603074552151
i=3 starts
fit=0.8714444603123669
i=4 starts
fit=0.849271196122048
i=5 starts
fit=0.8847680360515903
i=6 starts
fit=0.7268012267216293
i=7 starts
fit=0.7526922520325102
i=8 starts
fit=0.8475053908864587
i=9 starts
fit=0.8366281229237966
i=10 starts
fit=0.745290210317692
i=11 starts
fit=0.7951469786612839
i=12 starts
fit=0.7902546388207417
i=13 starts
fit=0.7634985452200084
i=14 starts
fit=0.7884699100716555
i=15 starts
fit=0.7359694677241304
i=16 starts
fit=0.7329274855142587
i=17 starts
fit=0.8072551227471396
i=18 starts
fit=0.7014844515958141
i=19 starts
fit=0.7590445084923547
i=20 starts
fit=0.7905921348228777
i=21 starts
fit=0.7973382282763977
i=22 starts
fit=0.761047988971835
i=23 starts
fit=0.7605742621169425
i=24 starts
fit=0.7618416897313299
i=25 starts
fit=0.48267763773499683
i=26 starts
fit=0.7491604905429154
i=27 starts
fit=0.32004187818070384
i=28 starts
fit=0.5421958211422

c:\Users\User\Anaconda3\envs\pytorch1.13\lib\site-packages\HydroErr\HydroErr.py:6248: UserWarning: Row(s) [1270 1271 4018 4019 4020 4021 4022 4023 4024 4025 4026 4027 4028 4029
 4030 4031 4032 4033 4034 4035 4036 4037 4038 4039 4040 4041 4042 4043
 4044 4045 4046 4047 4048 4049 4050 4051 4052 4053 4054 4055 4056 4057
 4058 4059 4060 4061 4062 4063 4064 4065 4066 4067 4068 4069 4070 4071
 4072 4073 4074 4075 4076 4077 4078 4079 4080 4081 4082 4083 4084 4085
 4086 4087 4088 4089 4090 4091 4092 4093 4094 4095 4096 4097 4098 4099
 4100 4101 4102 4103 4104 4105 4106 4107 4108 4109 4110 4111 4112 4113
 4114 4115 4116 4117 4118 4119 4120 4121 4122 4123 4124 4125 4126 4127
 4128 4129 4130 4131 4132 4133 4134 4135 4136 4137 4138 4139 4140 4141
 4142 4143 4144 4145 4146 4147 4148 4149 4150 4151 4152 4153 4154 4155
 4156 4157 4158 4159 4160 4161 4162 4163 4164 4165 4166 4167 4168 4169
 4170 4171 4172 4173 4174 4175 4176 4177 4178 4179 4180 4181 4182 4183
 4184 4185 4186 4187 4188 4189 4190 4191 4

fit=0.5972910071968319
i=254 starts
fit=-0.21321396766904344
i=255 starts
fit=0.4729049623408639
i=256 starts
fit=0.7051155563275576
i=257 starts
fit=0.2280305274386243
i=258 starts
fit=0.6948962616099452
i=259 starts
fit=0.6164929140246064
i=260 starts
fit=0.462609669805102
i=261 starts
fit=0.6311757021930104
i=262 starts
fit=0.7844595591335106
i=263 starts
fit=0.5827255529217394
i=264 starts
fit=0.570079475297335
i=265 starts
fit=0.5748585664633918
i=266 starts


KeyboardInterrupt: 

In [151]:
np.savetxt("data/ga_KGEs.csv", calibrated_KGES, delimiter=",")

#Recycle

In [96]:
def pred_hydrograph(code, x):

    record_length = x.shape[1]

    # iterate over each year for the selected catchments
    n_years = math.floor((record_length - BASE_LENGTH) / TARGET_SEQ_LENGTH)

    preds = torch.ones(record_length).to(computing_device)*torch.nan

    for k in range(n_years):
        start_record_ind = BASE_LENGTH + k * TARGET_SEQ_LENGTH
        end_record_ind = start_record_ind + TARGET_SEQ_LENGTH

        # dealing with the unique length of the last year record
        if k == (n_years - 1):
            end_record_ind = record_length

        # subsetting, base length is included
        x_sub = x[:, (start_record_ind - BASE_LENGTH) : end_record_ind, :]

        # pass through decoder, and store the result into `preds`
        with torch.autocast(
                device_type="cuda", dtype=torch.float16, enabled=use_amp
            ):
            with torch.no_grad():
                preds[start_record_ind:end_record_ind] = decoder.decode(code, x_sub)
        
    return(preds[BASE_LENGTH:])


class Objective_builder:
    def __init__(self, x, y, eval_fun):
        self.eval_fun = eval_fun
        self.x = x
        self.y = y
    
    def eval(self, code, return_summary = True):
        
        # numpy to torch tensor
        code = torch.from_numpy(code).unsqueeze(0).to(dtype=torch.float32).to(computing_device)
        
        # BASE_LENGTH is from global
        pred = pred_hydrograph(code,self.x).cpu().detach().numpy()
        ob = self.y[:,BASE_LENGTH:].squeeze().cpu().detach().numpy()
        
        if return_summary:
             return self.eval_fun(simulated_array=pred, observed_array=ob)
        else:
            return pred, ob

In [143]:
num_generations = 200
num_parents_mating = 10

sol_per_pop = 100
num_genes = LATENT_dim

init_range_low = catchment_embeddings.detach().cpu().min().numpy().tolist()
init_range_high = catchment_embeddings.detach().cpu().max().numpy().tolist()

parent_selection_type = "sss"

crossover_type = "single_point"

mutation_type = "random"
mutation_probability = 0.25

def evaluate_calibration(selected_catchment=[0]):
    
    x = dtrain.x[selected_catchment]
    y = dtrain.y[selected_catchment]

    x, y = x.to(computing_device), y.to(computing_device)

    fn = Objective_builder(x,y,HydroErr.kge_2009)

    def fitness_func(solution, solution_idx):
        return fn.eval(solution)

    ga_instance = pygad.GA(num_generations=num_generations,
                        num_parents_mating=num_parents_mating,
                        fitness_func=fitness_func,
                        sol_per_pop=sol_per_pop,
                        num_genes=num_genes,
                        init_range_low=init_range_low,
                        init_range_high=init_range_high,
                        parent_selection_type=parent_selection_type,
                        crossover_type=crossover_type,
                        mutation_type=mutation_type,
                        mutation_probability = mutation_probability,
                        stop_criteria=["saturate_10"])

    ga_instance.run()

    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    
    # evaluate on test dataset
    x = dtest.x[selected_catchment]
    y = dtest.y[selected_catchment]

    x, y = x.to(computing_device), y.to(computing_device)

    fn = Objective_builder(x,y,HydroErr.kge_2009)

    return fn.eval(solution)


In [144]:
N_CATCHMENTS = 1
calibrated_KGES = np.ones(N_CATCHMENTS)

for i in range(N_CATCHMENTS):
    print(f'i={i} starts')
    calibrated_KGES[i] = evaluate_calibration([i])
    print(f'fit={calibrated_KGES[i]}')

i=0 starts
fit=0.678217426649486


In [ ]:
calibrated_KGES

array([ 0.64549614,  0.81204675,  0.68903169,  0.74558772,  0.7675922 ,
        0.7616183 ,  0.52261648,  0.62396041,  0.6380763 ,  0.47799014,
        0.44281053,  0.5669716 ,  0.74182468,  0.69476666,  0.59849618,
        0.49235312,  0.55746592,  0.59070129,  0.61394915,  0.64278657,
        0.64090042,  0.61303593,  0.56211955,  0.75981612,  0.57187966,
        0.34825434,  0.64875829, -0.61393303,  0.41343837,  0.38551641,
        0.54683489,  0.6278619 ,  0.47030555,  0.53165741,  0.2726312 ,
        0.67005253,  0.44225278,  0.68713284, -0.35170959,  0.43675859,
        0.78642035,  0.10679402,  0.61263978,  0.58562663,  0.72905062,
        0.66690109,  0.80273776,  0.80054834,  0.6785977 ,  0.54791335,
        0.59791749,  0.71845921,  0.31151048,  0.42241099,  0.68594107,
        0.66256207,  0.64189628,  0.57072478,  0.64268098, -0.04243907,
       -0.26548802,  0.03542971,  0.0231844 ,  0.59270909,  0.6440923 ,
        0.51644761,  0.56432002,  0.47277055,  0.35394371,  0.50

In [ ]:
calibrated_KGES

array([ 3.68465379e-01,  4.30813003e-01,  3.84544370e-01,  3.92909455e-01,
        4.03020138e-01,  2.52027951e-01,  2.32128449e-01,  2.66625153e-01,
        4.39365295e-01,  4.90271430e-01,  3.72791553e-01,  3.71478866e-01,
        3.99828770e-01,  4.45838943e-01,  3.97063941e-01,  3.79567516e-01,
        3.39348909e-01,  4.58646336e-01,  4.74134986e-01,  5.06719075e-01,
        4.11992438e-01,  4.96153703e-01,  3.36959656e-01,  3.44994127e-01,
        5.54788634e-01,  3.94507039e-01,  4.41753713e-01, -2.90710820e-01,
        4.35333716e-01,  5.30392328e-01,  4.05321112e-01,  3.94015251e-01,
        4.61384573e-01,  4.59759621e-01,  3.30386720e-01,  3.17419376e-01,
        3.19606444e-01,  2.29067859e-01,  1.82640739e-01,  1.12859334e-01,
        3.67037467e-01, -5.02547369e-02,  1.35684418e-01,  2.66136003e-01,
        2.93613560e-01,  2.45347126e-01,  2.71608156e-01,  2.42388020e-01,
        2.05825991e-01,  1.26837906e-01,  8.71323680e-02,  1.85366497e-01,
        2.07907745e-01,  

In [ ]:
calibrated_KGES.max()

0.9194417028829057

In [ ]:
calibrated_KGES.size

546

In [ ]:
evaluate_calibration([2])

0.7892894999233896

In [ ]:
np.savetxt("data/ga_KGEs.csv", calibrated_KGES, delimiter=",")

In [ ]:
plt.hist(calibrated_KGES[calibrated_KGES>0], bins = 30)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
calibrated_KGES.mean()

0.44745825124666394

In [ ]:
np.median(calibrated_KGES)

0.5376689670561398

In [ ]:
N_CATCHMENT = train_val_batch_gen.dataset.x.shape[0]
calibrated_KGES = np.ones(N_CATCHMENT)
i = 0
calibrated_KGES[i] = evaluate_calibration([i])

In [ ]:

selected_catchment = [110]

batch_gen = train_val_batch_gen

x = batch_gen.dataset.x[selected_catchment]
y = batch_gen.dataset.y[selected_catchment]

x, y = x.to(DEVICE), y.to(DEVICE)

In [ ]:
fn = FN(x,y,HydroErr.kge_2009)

def fitness_func(solution, solution_idx):
    return fn.eval(solution)

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_probability = mutation_probability)

In [ ]:
ga_instance.run()

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [ 2.39297483 -3.06968491 -2.88818786 -2.89980175]
Fitness value of the best solution = 0.5666582062553918


In [ ]:
batch_gen = test_batch_gen

x = batch_gen.dataset.x[selected_catchment]
y = batch_gen.dataset.y[selected_catchment]

x, y = x.to(DEVICE), y.to(DEVICE)

fn = FN(x,y,HydroErr.kge_2009)

fn.eval(solution)

0.6630948503353136

In [ ]:
scaler = 1.25

UB, _ = torch.max(
    embedding.weight,
    0,
)

LB, _ = torch.min(
    embedding.weight,
    0,
)

UB = UB + torch.abs(UB)*(scaler - 1)
LB = LB - torch.abs(LB)*(scaler - 1)

UB = UB.cpu().detach().numpy()
LB = LB.cpu().detach().numpy()